# Функции потерь для задач классификации в PyTorch

Основной функцией потерь для задач классификации является *перекрестная энтропия* (cross-entropy):  

$$H(p,q)=-\sum_{y}p(y)\log q(y),$$
где $p$ – истинное вероятностное распределение, $q$ – вероятностное распределение, предсказанное моделью.  

В PyTorch существует несколько вариантов [функций потерь](https://pytorch.org/docs/stable/nn.html#loss-functions), связанных с перекрестной энтропией, основными из которых являются следующие:
- [BCELoss](#Binary-Cross-Entropy-(BCELoss)) – перекрестная энтропия для бинарной классификации;
- [CrossEntropyLoss](#Cross-Entropy-(CrossEntropyLoss)) – перекрестная энтропия для многоклассовой классификации;
- [NLLLoss](#Negative-Log-Likelihood-(NLLLoss)) – отрицательное логарифмическое правдоподобие (negative log likelihood).

In [ ]:
import numpy as np
import scipy.special
import torch 
import torch.nn as nn

precision = 8 # Точность после запятой

## Binary Cross Entropy (BCELoss)

*Бинарная перекрестная энтропия* (Binary Cross-Entropy):  

$$L = -\frac{1}{l}\sum_{i=1}^{l}y_i\log(p(y_i))+(1-y_i)\log(1-p(y_i)),$$

где $y_i$ – истинный ответ, $y_i\in\{0,1\}$;  
$p(y_i)$ – предсказанная вероятность ответа $y_i=1$.  

Применяется для задач бинарной классификации.

#### Вычисление в NumPy
Обратите внимание на размерности массивов данных – это одномерные массивы.

In [ ]:
# Два класса: [0, 1]
# Три примера
# Числа обозначают вероятности отнесения примера к классу 1
y_pred = np.array([0.1, 0.9, 0.2])
y_true = np.array([0.0, 1.0, 0.0])

In [ ]:
# Binary Cross-Entropy
def BCE(y_pred, y_true):
    total_bce_loss = -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    num_of_samples = y_pred.shape[0]
    mean_bce_loss = total_bce_loss / num_of_samples
    return mean_bce_loss

In [ ]:
bce_loss_np = BCE(y_pred, y_true)
print(f'BCE loss (NumPy) = {bce_loss_np:.{precision}f}')

#### Вычисление в PyTorch

In [ ]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [ ]:
bce_loss_fn = nn.BCELoss()
bce_loss_torch = bce_loss_fn(y_pred_tensor, y_true_tensor)
print(f'BCE loss (PyTorch) = {bce_loss_torch:.{precision}f}')

## Cross Entropy (CrossEntropyLoss)
Перекрестная энтропия в случае многоклассовой классификации (Categorical Cross-Entropy, Multiclass Cross-Entropy):  

$$L = -\frac{1}{l}\sum_{i=1}^{l}y_{i}\log(p(y_{i}))$$

В PyTorch в функции `CrossEntropyLoss()` предсказанные значения считаются ненормализованными, поэтому перед вычислением перекрестной энтропии к ним применяется функция`Softmax`.

#### Вычисление в NumPy
На входе двумерные массивы: количество строк совпадает с количеством примеров, количество столбцов – с количеством классов.  
В каждой строке находятся вероятности принадлежности данного примера к классу, номер которого соответствует индексу столбца (от 0 до K–1).

In [ ]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

In [ ]:
# Применяем Softmax, как в PyTorch
y_pred_softmax = scipy.special.softmax(y_pred, axis=1)
y_pred_softmax

In [ ]:
def CrossEntropy(y_pred, y_true):
    total_loss = -np.sum(y_true * np.log(y_pred))
    num_of_samples = y_pred.shape[0]
    mean_loss = total_loss / num_of_samples
    return mean_loss

In [ ]:
cross_entropy_loss_np = CrossEntropy(y_pred_softmax, y_true)
print(f'Cross Entropy loss (NumPy) = {cross_entropy_loss_np:.8f}')

#### Бинарная классификация в NumPy (аналог BCELoss)
Для примера покажем вычисление бинарной перекрестной энтропии при помощи функции `CrossEntropy()`.  
Возьмем пример, который использовался выше в бинарной классификации.

In [ ]:
# Два класса: [0, 1]
# Три примера
# Числа обозначают вероятности отнесения примера к классу 1
# y_pred = np.array([0.1, 0.9, 0.2])
# y_true = np.array([0.0, 1.0, 0.0])

# Преобразуем к двумерным массивам
y_pred = np.array([[0.9, 0.1], [0.1, 0.9], [0.8, 0.2]])
y_true = np.array([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]])

In [ ]:
cross_entropy_loss_np = CrossEntropy(y_pred, y_true)
print(f'Cross Entropy loss (NumPy) = {cross_entropy_loss_np:.8f}')

#### Вычисление в PyTorch
Вычисление многоклассовой перекрестной энтропии в PyTorch реализуется при помощи функции `CrossEntropyLoss()`.  
На вход функции поступает двумерный массив предсказанных вероятностей (строки – примеры, столбцы – классы) и одномерный массив истинных классов: значение в этом массиве обозначает класс (от 0 до K–1).

In [ ]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

# Преобразуем двумерный массив y_true в одномерный
y_true = np.array([0, 1, 2, 0]).astype(np.int64) # номера классов из диапазона [0, K-1]

In [ ]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [ ]:
cross_entropy_loss_fn = nn.CrossEntropyLoss()
cross_entropy_loss_torch = cross_entropy_loss_fn(y_pred_tensor, y_true_tensor)
print(f'Cross Entropy loss (PyTorch) = {cross_entropy_loss_torch:.8f}')

#### Бинарная классификация в PyTorch (аналог BCELoss)
Также для примера продемонстрируем вычисление бинарной перекрестной энтропии при помощи функции PyTorch `CrossEntropyLoss()`.
Возьмем пример, который использовался выше в бинарной классификации.

In [ ]:
# Два класса: [0, 1]
# Три примера
# Числа обозначают вероятности отнесения примера к классу 1
# y_pred = np.array([0.1, 0.9, 0.2])
# y_true = np.array([0.0, 1.0, 0.0])

# Преобразуем к двумерным массивам
y_pred = np.array([[0.9, 0.1], [0.1, 0.9], [0.8, 0.2]])
y_true = np.array([0, 1, 0]).astype(np.int64) # номера классов из диапазона [0, K-1]

In [ ]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true) 

In [ ]:
# Логарифмируем, чтобы компенсировать Softmax, встроенный в CrossEntropyLoss()
y_pred_log = torch.log(y_pred_tensor)

In [ ]:
cross_entropy_loss_fn = nn.CrossEntropyLoss()
cross_entropy_loss_torch = cross_entropy_loss_fn(y_pred_log, y_true_tensor)
print(f'Cross Entropy loss (PyTorch) = {cross_entropy_loss_torch:.8f}')

## Negative Log Likelihood (NLLLoss)
*Отрицательное логарифмическое правдоподобие* (Negative Log Likelihood) – это другое название перекрестной энтропии.  
В PyTorch отрицательное логарифмическое правдоподобие реализуется при помощи функции `NLLLoss()`, но предполагается, что предсказанные ненормализованные значения $x_i$ преобразуются с помощью функции `LogSoftmax()`:

$$LogSoftmax(x_i)=\log\left(\frac{e^{x_i}}{\sum_{j}x_j}\right)$$

Поэтому в PyTorch перед использованием `NLLLoss()` ненормализованные выходы следует преобразовать с использованием функции `LogSoftmax()`.

#### Вычисление в NumPy

In [ ]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

In [ ]:
# Применяем Softmax
y_pred_softmax = scipy.special.softmax(y_pred, axis=1)
y_pred_softmax

In [ ]:
# Применяем логаримф - получается LogSoftmax
y_pred_log_softmax = np.log(y_pred_softmax)
y_pred_log_softmax

In [ ]:
def NLLLoss(y_pred, y_true):
    # То же, что CrossEntropy, но без логарифма перед y_pred
    total_loss = -np.sum(y_true * y_pred)
    num_of_samples = y_pred.shape[0]
    mean_loss = total_loss / num_of_samples
    return mean_loss

In [ ]:
nll_loss_np = NLLLoss(y_pred_log_softmax, y_true)
print(f'Negative Log Likelihood loss (NumPy) = {nll_loss_np:.8f}')

#### Вычисление в PyTorch

In [ ]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

# Преобразуем двумерный массив y_true в одномерный
y_true = np.array([0, 1, 2, 0]).astype(np.int64) # номера классов из диапазона [0, K-1]

In [ ]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [ ]:
softmax = nn.LogSoftmax(dim=1)
y_pred_tensor = softmax(y_pred_tensor)

In [ ]:
nll_loss_fn = nn.NLLLoss()
nll_loss_torch = nll_loss_fn(y_pred_tensor, y_true_tensor)
print(f'Negative Log Likelihood loss (PyTorch) = {nll_loss_torch:.8f}')

## Резюме
Обозначим `y` – ненормированный выход последнего слоя нейронной сети (например, выход модуля `Linear`).
1. Бинарная классификация:
    - `y` 🡒 `Sigmoid` 🡒 `BCELoss`
1. Многоклассовая классификация:
    - `y` 🡒 `LogSoftmax` 🡒 `NLLLoss`
    - `y` 🡒 `CrossEntropyLoss` (аналог: `y` 🡒 `LogSoftmax` 🡒 `NLLLoss`)

Если требуется на выходе сети явно иметь результаты `Softmax`, можно использовать следующую схему:  
- `y` 🡒 `Softmax` 🡒 `Log` 🡒 `NLLLoss`